In [12]:
import numpy as np
import os
from astropy.io import fits
from astropy.convolution import Gaussian2DKernel
base = "/home/max/Development/semester-project/data/"

# Multi Mode Source Extraction for the James Webb Space Telescope

----
_Massimo Bortone, Institute of Particle Physics and Astrophysics, ETH Zürich_
![JWST](images/jwst-nasa.jpeg "JWST")
- _Supervisors at Harvard-Smithsonian Center for Astrophysics: Dr. Sandro Tacchella, Prof. Daniel Eisenstein, Dr. Ben Johnson_
- _Supervisor at ETH Zürich: Prof. Alexandre Refregier_

# Outline
----
1. Motivations & Objectives
2. SExtractor
3. Overshredding
4. Kernels
5. Multi Mode Source Extraction
6. Next steps

# Motivations
----
* Overlapping sources in deep images are a problem for photometry tools
* Need for realistic uncertainties and degeneracies in photometric measurements
* Use information from all bands of the NIRCam instrument on JWST

- Aperture photometry methods like *Kron's first moment* algorithm fail when sources are overlapping
- Better estimates of photometric redshifts, colors and color gradients are need to accurately study the evolution of early galaxies
- NIRCam will allow detection of galaxy at redshifts possibly beyond $z>15$, important to include multiple bands do detect dropouts

# Objectives
----
* Detect as many possible sources in astronomical images of deep fields
* Identify sub-structure components in galaxies (*overshred detection*)
* Combine detections from multiple bands
* Forward model the image with `FORCEPho` to obtain probabilistic assignments of fluxes

# The HXDF
----
<div style="width: 100%;">
    <div style="float: left; width: 50%">
        <img src="images/xdf.png" alt="XDF and cutout region" />
    </div>
    <ul style="float: left; width: 45%">
        <li>$20 ~\text{days}$ total exposure</li>
        <li>$2.3\times 2 ~\text{arcminutes}^2$ in size, which is about $\frac{1}{32\times 10^6}$ of the sky</li>
        <li>generated using data from Hubble's ACS/WFC and WFC3/IR cameras</li>
        <li>deepest image of the sky in the optical/near-IR regime</li>
        <li>about 5500 galaxies, with redshift $z<10$</li>
    </ul>
</div>

## Multiple bands
----
<div style="width: 100%;">
    <div style="float: left; width: 70%">
        <img src="images/H-I-bands.png" alt="H and I bands of the XDF" />
    </div>
    <ul style="float: left; width: 27%">
        <li>detect dropouts</li>
        <li>improve detection of sub-structure components</li>
        <li>H band: F160W, 1600nm</li>
        <li>I band: F814W, 814nm</li>
    </ul>
</div>

![Pipeline Overview](images/pipeline-overview.jpeg "Data-Analysis Pipeline Overview")

# FORCEPho
----
<div style="width: 100%;">
    <div style="float: left; width: 50%">
        <img src="images/forcepho-example.png" alt="FORCEPho example"/>
    </div>
    <ul style="float: left; width: 47%">
        <li>Gaussian mixtures for sources and PSF</li>
        <li>2nd-order approximation to pixel integral of a Gaussian to fit undrizzled, undersampled exposures</li>
        <li>Sersic profiles with 6 parameters to model galaxies, approximated by GMs</li>
        <li>Hamiltonian-Monte-Carlo optimization of $\chi^2$ ln-likelihood</li>
    </ul>
</div>

# SExtractor
----
Developed in 1996 by E. Bertin and S. Arnouts, is a signal-based method to extract sources from large astronomical images.

Fast and based on well motivated heuristic, but requires good tuning of the parameters.

## How it works
----
The complete analysis of an image consists of 6 steps:
1. estimation of sky background
2. **thresholding**
3. **deblending**
4. cleaning
5. photometry
6. star/galaxy separation

1. assume image is already background subtracted or background is sufficiently flat
4. not relevant for our goals
5. handled by FORCEPho
6. not important in deep images

## Thresholding
----
* convolve original image with a filtering kernel 
* thresholding is local for every pixel with position $(x,~y)$:
$$I_{ij} > \sigma \cdot \sqrt{V_{ij}},~\forall (i,~j) \in G= \{(x \pm 1,~y), (x,~y\pm 1), (x,~y\pm 1)\}$$
where $I_{ij}$ and $V_{ij}$ are the intensity and the variance of the pixel at position $(i, j)$ on the image, while $\sigma$ is the detection threshold
* extract 8-connected contigous pixels above threshold from detection image

## Deblending
----
* re-threshold each extracted set of 8-connected pixels at `NTHRESH` exponentially spaced levels between the primary extraction threshold $\sigma$ and the peak value in the extracted set
* model light distribution of the object as a tree structure
* `MINCONT` parameter defines the minimal contrast $\delta_c$ between the integrated pixel density in the branches and the total density of the composite object

**CONDITIONS FOR SPLITTING**:

<div style="width: 100%;">
    <ol style="float: left; width: 50%; margin-top: 40px;">
        <li>$\frac{\text{integrated pixel density in branch}}{\text{total density}} > \delta_c$</li>
        <li>the first condition is true for another branch at the same level</li>
    </ol>
    <div style="float: left; width: 45%">
        <img src="images/sextractor-deblending.png" alt="Deblending in SExtractor" />
    </div>
</div>

# Overshredding
----
Apart from $\sigma$, the detection behavior of SExtractor is controlled by
- `MINAREA`: minimal number of pixels that define an object
- `NTHRESH`: number of exponentially spaced levels
- `MINCONT`: minimal contrast needed to split an object in two components

- `MINAREA` is quite straightforward: the smaller, the more "objects" are detected
- set `MINAREA` to 5 pixels, corresponds to about 1 FWHM of the PSF or about $0.18\times 0.18~\text{arcseconds}^2$
- not clear which values of the other parameters are good for our goals

## Exploring the parameter space
----
Explore the parameter space to find which combination of values of `NTHRESH` and `MINCONT` yield to overshredding

In [ ]:
nthresh = range(32, 1025, 32)
mincont = np.logspace(0, -10, 100)

![grid](images/mincont-nthresh-grid.png "Detections as a function of NTHRESH and MINCONT")

**1. CONCLUSION** 

`MINCONT` has a larger effect on the number of detections than `NTHRESH`

![grid](images/detection-vs-mincont.png "Detections as a function MINCONT for NTHRESH=64")

**2. CONCLUSION** 

Number of detections saturates for `MINCONT`$<10^{-6}$

--------
From now on we fix the parameters to:
- `NTHRESH`=$64$
- `MINCONT`=$10^{-7}$

# Kernel
----
A kernel is a matrix of a given size that is convolved with the image to increase the sensitivity of the detection towards specific objects

## Examples

* PSF kernel: good for faint unresolved objects
* flat kernel: better for LSB objects

In [7]:
psf_filename = os.path.join(base, "ZC404221_H_PSF.fits")
psf_hdu = fits.open(psf_filename)
psf = psf_hdu[0].data
psf_clipped = np.array(psf[40:60, 40:60])

PSF has FWHM of 3 pixels, which corresponds to an area of $0.18\times 0.18 ~\text{arcseconds}^2$

In [8]:
psf_default = np.zeros_like(psf_clipped)
psf_default[9:12, 9] = [1, 2, 1]
psf_default[9:12, 10] = [2, 4, 2]
psf_default[9:12, 11] = [1, 2, 1]
psf_default /= 4.0

In [9]:
flat_5 = np.zeros_like(psf_clipped)
flat_5[8:13, 8:13] = np.ones((5, 5))

In [10]:
one_hot = np.zeros_like(psf_clipped)
one_hot[10, 10] = 1

![Kernels](images/kernels.png "4 different kernels")

Comparing the number of detections as a function of area for the 4 different kernels we notice:

![Histograms](images/hist-kernels.png "Histograms of detections for 4 different kernels")

- `ONE-HOT` kernel detects a lot of small scale structure
- `PSF` and `FLAT` kernels are sensible to more extended objects
- about 1 order of magnitude difference in average area of object detected

Overplotting all detections we find:
![All detections](images/all-kernels.png "Detected sources for 4 different kernels")

- SExtractor with the `ONE-HOT` kernel can be use to detect substructures in bright extended objects
- broad kernels should be used to detect bright objects, then replacing them with sub-structure components from narrow kernels

# Multi Mode Source Extractor
----
**IDEA:** 
run SExtractor in different modes and on different bands, allowing each mode to specify its own set of SExtractor parameters. Then merge detections into one single catalog.

----
**PROBLEM:** 
merging detections from multiple modes and bands is not straightforward

## Defining modes and bands
----

In [1]:
kernel_cold = np.asarray(Gaussian2DKernel(3))
kernel_hot = np.zeros((3,3))
kernel_hot[1,1] = 1

NameError: name 'np' is not defined

In [14]:
modes = {
    'H': {
        # detect large bright sources
        'cold': {
            'sigma': 1.5,
            'mincont': 1e-7,
            'nthresh': 64,
            'kernel': kernel_cold,
            'minarea': 5
        },
        # detect small isolated sources
        'hot': {
            'sigma': 1.5,
            'mincont': 1e-7,
            'nthresh': 64,
            'kernel': kernel_hot,
            'minarea': 5
        }
    },
    'I': {
        # detect substructure in large bright sources
        'hot': {
            'sigma': 2.0,
            'mincont': 1e-7,
            'nthresh': 64,
            'kernel': kernel_hot,
            'minarea': 5
        }
    }
}

In [19]:
bands = {
    'H': {
        'image': image_h,
        'rms': rms_h
    },
    'I': {
        'image': image_i,
        'rms': rms_i
    }
}

## Finding substructures
----
The idea is to use a small kernel on the I band to detect sub-structures within the brightest objects detected in the H band. To do this we need to:

1. Compute fluxes for all objects
2. Select brightest objects in the H band
3. Look for sub-structures in the I band

![H and I detections](images/H-I-detections.png "Detections in the H and I bands")

### 1. Compute flux within elliptical aperture for each detection

$$
\texttt{CXX}(x-\bar{x})^2 + \texttt{CYY}(y-\bar{y})^2 + \texttt{CXY}(x-\bar{x})(y-\bar{y}) = R^2
$$
![Integrated flux](images/H-integrated-flux.png "Integrated flux for detections in the H band")

- CXX, CYY, CXY are computed by SExtractor from the object's light distribution second moments

### 2. Select top 20% brightest objects in the H band
![Top 20% detections](images/H-flux-selection.png "Top 20% brightest detections in the H band")

### 3. Search for detections in the I band that are within an elliptical aperture of 2x the dimensions determined by SExtractor
![I Sub-structures](images/I-band-substructure.png "Sub-structure detection in the I band")

### 4. Repeat same steps to find sub-structures on the H band
![H Sub-structures](images/H-band-substructure.png "Sub-structure detection in the H band")

### 5. Compare sub-structures in both bands
![Compare sub-structures](images/H-vs-I-substructure.png "Comparison between substructure detected in the H and I bands")

## Merging
----
In order to merge the detections obtained for the different bands and modes we need to:

1. Remove duplicates across bands and modes
2. Remove brightest objects in H band
3. Replace sub-structures in H band with those found in the I band

### Finding similar detections
----
Given two objects $V$ and $W$, we define a proximity function as follows:

$$
 P(V, W) = \sqrt{(\bar{x}_V - \bar{x}_W)^2 + (\bar{y}_V - \bar{y}_W)^2} + \vert s_V - s_W \vert + \vert \theta_V - \theta_W \vert
$$

where $(\bar{x}_i,~\bar{y}_i)$ are the center coordinates, $s_i = a_i \cdot b_i$ is the surface of the ellipse estimated from the light distribution and $\theta_i$ is its orientation with respect to the horizontal axis.

Pairs of similar objects are those for which $P(V, W)$ is below a certain user-defined threshold

### 1.1 Remove duplicates across modes in the H band

![Duplicates H band](images/H-band-hot-cold-similar.png "Duplicates across modes in the H band")

### 1.2 Remove duplicates across H and I bands in hot mode

![Duplicates hot mode](images/H-I-band-hot-hot-similar.png "Duplicates across H and I bands in hot mode")

## Final positions
----
![Final posititons](images/final-positions.png "Final positions")

- sub-structure detection (*overshredding*) is effective for bright objects
- combining multiple bands helps detecting dropouts
- spurious detections in bright halos around large objects need to be further confirmed, maybe using photometry information from `FORCEPho`

## Comparison to other catalogs
----
We compare our detections to those found by:
- Bouwens+ 2015 (see http://vizier.cfa.harvard.edu/viz-bin/VizieR?-source=J/ApJ/803/34)
- Guo+ 2013 (CANDELS GOODS-S catalog, see http://vizier.cfa.harvard.edu/viz-bin/VizieR?-source=J/ApJS/207/24)
- Finkelstein+ 2015 (see http://vizier.cfa.harvard.edu/viz-bin/VizieR?-source=J/ApJ/810/71)
- Skelton+2014 (3D-HST catalog, see http://adsabs.harvard.edu/abs/2014ApJS..214...24S)

Source extraction methods:

- Bouwens and Finkelstein are optimized towards detection of high-redshift galaxies ($z>4$)
- Bouwens stacks $Y_{098}Y_{105}J_{125}H_{160}$ WFC3/IR observations
- Finkelstein adopts weighted sum of F125W and F160W images
- Guo uses a similar strategy to ours with hot and cold source extraction modes
- 3D-HST uses coadded F125W + F140W + F160W images

Overplotting all detections we find:
    
![All catalogs](images/all-mmse.png "Detections from all catalogs")

**Results**:

![Detection fraction](images/detection-recovery.png "Detection fraction")

### Stacking
----
Agreement with `Bouwens` and `Finkelstein` catalogs can be improved by stacking YJH bands together.

![Stacked](images/Hstacked-mmse.png "Stacked H band detections")

# What's next
----

- measure distance between peak and barycentre in each detected object to determine the width of a prior on its final position
- determine detection recovery as a function of confusion by introducing artificial objects on the image
- using simulated images:
    - quantify false positive and true negative detections
    - find optimal kernels for different kinds of objects

- for points that close to each other, need to possibly fix one in order to avoid source-swapping in FORCEPho
- eliminate spurious detections by commparing FORCEPho model with initial detections, remove those with zero posteriors